In [1]:
#run this cell
import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os


import gymnasium as gym


import collections
import argparse

import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import seaborn as sns
import pandas as pd
from collections import defaultdict

In [2]:
import torch
import torch.utils.data as utils
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torch.optim as optim


from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
main_data_list = ['main_matrix1.npz', 'main_matrix2.npz', 'main_matrix3.npz']

In [8]:
test = np.load('main_matrix1.npz')

In [ ]:
with np.load('main_matrix1.npz') as data:
    matrix

In [15]:
test['matrix']

KeyError: 'matrix is not a file in the archive'

In [19]:
matrix1 = list(test.values())

In [21]:
matrix1[0]

array([[[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [29]:
matrix_main = []
for i in main_data_list:
    matrix = np.load(i)
    matrix = list(matrix.values())
    
    matrix_main.append(matrix[0])

In [30]:
matrix_main[0]

array([[[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [31]:
matrix_main[1]

array([[[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [32]:
matrix = np.concatenate(matrix_main, axis=0)

In [33]:
matrix.shape

(62223, 1553, 17)

In [35]:
torch.tensor(matrix)

tensor([[[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

: 

In [34]:
matrix

array([[[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [50]:
data = torch.load('main_tensor.pt')
data = data.cpu().numpy()

In [74]:
data.shape[0] / 3

20741.0

In [75]:
main_data_list = ['main_matrix1', 'main_matrix2', 'main_matrix3']



In [76]:
matrix1 = data[:20741]
matrix2 = data[20741 : 20741 * 2]
matrix3 = data[20741 * 2 : 20741 * 3]

In [78]:
20741 * 2

41482

In [79]:
20741 * 3

62223

In [80]:
np.savez_compressed('main_matrix1.npz', matrix1)
np.savez_compressed('main_matrix2.npz', matrix2)
np.savez_compressed('main_matrix3.npz', matrix3)

In [ ]:
np.savez_compressed('test.npz', dummy)

In [72]:
data

array([[[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [73]:
np.savez_compressed('test2.npz', data)

In [51]:
last_matrix = data[60000:]
last_matrix

array([[[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [53]:
current_matrix = data[:60000]

In [65]:
window =  current_matrix.shape[0] / 30
window

2000.0

In [69]:
dummy = data[:3000]

In [70]:
dummy

array([[[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [71]:
np.savez_compressed('test.npz', dummy)

In [68]:
np.save('test.npy', dummy)

In [48]:
data.shape[0] - 2223

60000

In [46]:
for i in range(2,200):
   if (data.shape[0] / i) % 10 == 1:
        print(i)


3


In [41]:
10 / 4 % 10

2.5

In [11]:
data[:8889].shape

torch.Size([8889, 1553, 17])

In [31]:
dummy = data[:2963]

In [32]:
dummy = dummy.cpu().numpy()

In [13]:
torch.save(dummy, 'test.pt')

In [31]:
data = torch.load('main_tensor.pt').cpu().numpy()
data

array([[[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [26]:
target = torch.load('target.pt')
index = torch.load('index.pt')


target = target.cpu().numpy()
index = index.cpu().numpy()

In [33]:
target

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [34]:
target.sum()

205.0

In [36]:
new = (1 - target)
new.sum()

14863370.0

In [13]:
current_step = 0

In [18]:
index[current_step]

array([  0, 188], dtype=int64)

In [27]:
first_login = index[current_step][0]
last_login = index[current_step][1]

In [28]:
last_login

188

In [29]:
observation = data[first_login : last_login]
target_observation = target[first_login :last_login]
observation

tensor([[[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [30]:
target_observation

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

In [32]:
target_observation.sum()

0.0

In [12]:
index[2]

tensor([364, 576])

In [5]:
ten

tensor([[[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         [1., 2., 3.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [7]:
import cachetools

In [3]:
import redis

In [12]:
import pickle

In [9]:
# r = redis.Redis(host='localhost', port=6379, db=0)

In [8]:
temp = pd.read_feather('new.feather')

In [6]:
temp.to_pickle('main.pkl')

In [7]:
del temp

In [9]:
serialized_df = pickle.dumps(temp)

In [10]:
r.set('my_data_key', 'main.pkl')

True

In [14]:
serialized_df = r.get('my_data_key')
# df = pickle.loads(serialized_df)

In [15]:
type(serialized_df)

bytes

In [ ]:
pd.read_pickle()

In [ ]:
pd.re

In [ ]:
from DataManager import ZipDataMerger, FileMaker
from Data import BaseDataReader,TimeStep, SimpleTokenizer
import re

In [ ]:
env = TimeStep()
tokenizer = SimpleTokenizer()
df = env.base_data.copy()

temp = df.copy()
df = df.drop(['target'],axis=1)

In [ ]:
targets = temp['target'].values

In [ ]:
target_array = []
for i in df.values:
    result = " ".join(i)
    target_array.append(result)

In [ ]:
tokens = []
for i in target_array:
    tok = re.findall(r'\w+|\S', i)
    tokens.append(tok)

df['tokens'] = tokens

In [ ]:
df['new_tokens'] = df['tokens'].apply(tokenizer.convert_tokens_to_ids)

In [ ]:

padded_tokens = []
for i in df['new_tokens'].values:
    padded_array = np.zeros(17)
    length_int = len(i)
    padded_array[:length_int] = i
    padded_tokens.append(padded_array)

In [ ]:
df['total_padded_tokens'] = padded_tokens
df.to_feather('new.feather')

In [ ]:
temp = pd.read_feather('new.feather')

In [ ]:
df = temp[['total_padded_tokens']]
del temp

In [ ]:
data = df['total_padded_tokens'].values

In [ ]:
observation_data = np.vstack(data)

In [ ]:
#and this cell
matrices_list = []
env.reset()
window_size = env.index_list[env.current_step]
# window = env.data[window_size[0]:window_size[1]]

for i in env.index_list[:-1]:
    
    observation = observation_data[i[0] : i[1]]
    matrix = np.zeros((1553, 17))

    
    matrix[:observation.shape[0], :] = observation
    env.array_step()
    matrices_list.append(matrix)
    window_size = env.index_list[env.current_step]

In [ ]:
larger_list = np.array(matrices_list)

In [ ]:
login_tensors = torch.tensor(larger_list, dtype=torch.float32)
login_tensors